In [1]:
library(ISLR)
library(boot)

In [2]:
# Validation-Set 
## Split: Takes a sample of the specified size from the original data using either with or without replacement
set.seed(1)
train = sample(392, 196)
## Fit: Runs a linear regression using only the training observations 
lr_model_1 = lm(mpg ~ horsepower, data = Auto, subset = train)
lr_model_2 = lm(mpg ~ poly(horsepower, 2), data = Auto, subset = train)
lr_model_3 = lm(mpg ~ poly(horsepower, 3), data = Auto, subset = train)
## Validate: Calculates test error rate 
attach(Auto)
mean((mpg - predict(lr_model_1, Auto))[-train] ^ 2)
mean((mpg - predict(lr_model_2, Auto))[-train] ^ 2)
mean((mpg - predict(lr_model_3, Auto))[-train] ^ 2)

[1] 23.26601

[1] 18.71646

[1] 18.79401

In [3]:
# Leave-One-Out Cross-Validation 
## The 1st component: The raw cross-validation estimate of prediction error
## The 2nd component: The adjusted cross-validation estimate (Designed to compensate for the bias introduced by not using leave-one-out cross-validation)
loocv_err = rep(0, 5)
for (i in 1:5) {
    glm_model = glm(mpg ~ poly(horsepower, i), data = Auto)
    loocv_err[i] = cv.glm(Auto, glm_model)$delta[1]
}
print(loocv_err)

[1] 24.23151 19.24821 19.33498 19.42443 19.03321


In [4]:
# K-Fold Cross-Validation
kcv_err = rep(0, 10)
for (i in 1:10) {
    glm_model = glm(mpg ~ poly(horsepower, i), data = Auto)
    kcv_err[i] = cv.glm(Auto, glm_model, K = 10)$delta[1]
}
print(kcv_err)

 [1] 24.26262 19.23666 19.46337 19.42564 18.95917 18.94794 18.85494 18.82247
 [9] 19.22451 19.10879


In [5]:
# Bootstrap
# Use case 1: Estimate the accuracy of a statistic of interest
# Use case 2: Estimate the accuracy of a linear regression model

In [6]:
# Use case 1: Estimate the accuracy of a statistic of interest
## Create an estimate function 
alpha_fn = function(data, index) {
    X = data$X[index]
    Y = data$Y[index]
    return((var(Y) - cov(X, Y)) / (var(X) + var(Y) - 2 * cov(X, Y)))
}
## Perform a bootstrap process
boot(Portfolio, alpha_fn, R = 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha_fn, R = 1000)


Bootstrap Statistics :
     original        bias    std. error
t1* 0.5758321 -0.0009324929  0.09189317

In [7]:
# Use case 2: Estimate the accuracy of a linear regression model
## Create an estimate function 
lr_fn = function(data, index) {
    lr_model = lm(mpg ~ horsepower, data = data, subset = index)
    return(coef(lr_model))
}
## Perform a bootstrap process
boot(Auto, lr_fn, R = 1000)                      # SE of coefficient estimates obtained from bootstrap (Do not rely on certain assumptions) 
summary(lm(mpg ~ horsepower, data = Auto))$coef  # SE of coefficient estimates obtained from formulas (Do rely on certain assumptions)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = lr_fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0429311736 0.835516284
t2* -0.1578447 -0.0005363252 0.007242626

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),39.9358610,0.717498656,55.65984,1.220362e-187
horsepower,-0.1578447,0.006445501,-24.48914,7.031989e-81
